In [2]:
import importlib
import prepare_vocab
import train
import eval
import config
from data.loader import DataLoader
from utils import constant
importlib.reload(config)

<module 'config' from 'C:\\prabhu\\edu\\code\\w266\\final_project\\config.py'>

In [3]:

vocab_params = config.VocabParameters()
training_params = config.TrainingParameters()
eval_params = config.EvalParameters()
opt = vars(vocab_params)
opt['num_class'] = len(constant.LABEL_TO_ID)
opt.update(vars(training_params))


In [4]:
# %Load glove vectors, get IDs for each token, lookup for each id in embedings and create embeding file for out dataset.
vocab = prepare_vocab.prepare_voabulary (vocab_params)

loading files...
2525296 tokens from 68124 examples loaded from ./dataset/tacred/train.json.
802558 tokens from 22631 examples loaded from ./dataset/tacred/dev.json.
539009 tokens from 15509 examples loaded from ./dataset/tacred/test.json.
loading glove...
2195892 words loaded from glove.
building vocab...
vocab built with 55950/62152 words.
calculating oov...
train oov: 20546/2525296 (0.81%)
dev oov: 45801/802558 (5.71%)
test oov: 33634/539009 (6.24%)
building embeddings...
embedding size: 55950 x 300
dumping to files...
all done.


In [5]:
from utils.vocab import Vocab
vocab_file = vocab_params.vocab_dir + '/vocab.pkl'
vocab = Vocab(vocab_file, load=True)

Vocab size 55950 loaded from file


In [6]:
# load data
print("Loading data from {} with batch size {}...".format(vocab_params.data_dir, training_params.batch_size))
train_batch = DataLoader(vocab_params.data_dir+ '/train.json', training_params.batch_size, opt, vocab, evaluation=False)
dev_batch = DataLoader(vocab_params.data_dir + '/dev.json', training_params.batch_size, opt, vocab, evaluation=True)


Loading data from ./dataset/tacred with batch size 50...
1363 batches created for ./dataset/tacred/train.json
453 batches created for ./dataset/tacred/dev.json


In [7]:
# # Train model using Position Aware
# train.train_model(vocab_params, training_params, train_batch, dev_batch, model_id='00')

# # Train model using LSTM
# train.train_model(vocab_params, training_params, train_batch, dev_batch, model_id='01')

In [8]:
importlib.reload(config)
# eval_params.model_dir = 'C:\prabhu\edu\code\w266\tacred\saved_models\00\'

<module 'config' from 'C:\\prabhu\\edu\\code\\w266\\final_project\\config.py'>

In [9]:
import importlib
from utils import torch_utils 
importlib.reload(eval)
importlib.reload(torch_utils)
importlib.reload(config)
eval.evaluate_model(eval_params)

save_models/00 best.model.pt
Loading model from best_model.pt
Finetune all embeddings.
Vocab size 55950 loaded from file
Loading data from ./dataset/tacred/test.json with batch size 50...
311 batches created for ./dataset/tacred/test.json

Running with the following configs:
	data_dir : ./dataset/tacred
	vocab_dir : ./dataset/vocab
	glove_dir : ./dataset/glove
	emb_dim : 300
	vocab_file : /vocab.pkl
	embed_file : /embedding.npy
	glove_text_file : glove.840B.300d.txt
	lower : False
	min_freq : 0
	num_class : 42
	ner_dim : 30
	pos_dim : 30
	hidden_dim : 200
	num_layers : 2
	dropout : 0.5
	word_dropout : 0.04
	topn : 10000000000.0
	lower_dest : lower
	lower_action : store_true
	no_lower_dest : lower
	no_lower_action : store_false
	attn_dest : attn
	attn_action : store_true
	no_attn_dest : attn
	no_attn_action : store_false
	attn : True
	attn_dim : 200
	pe_dim : 30
	lr : 1.0
	lr_decay : 0.9
	optim : sgd
	num_epoch : 30
	batch_size : 50
	max_grad_norm : 5
	log_step : 20
	log : logs.txt
	sav

In [10]:
def guess_encoding(csv_file):
    """guess the encoding of the given file"""
    import io
    import locale
    with io.open(csv_file, "rb", encoding=locale.getdefaultlocale()[1]) as f:
        data = f.read(5)
    if data.startswith(b"\xEF\xBB\xBF"):  # UTF-8 with a "BOM"
        return "utf-8-sig"
    elif data.startswith(b"\xFF\xFE") or data.startswith(b"\xFE\xFF"):
        return "utf-16"
    else:  # in Windows, guessing utf-8 doesn't work, so we have to try
        try:
            with io.open(csv_file, encoding="utf-8") as f:
                preview = f.read(222222)
                return "utf-8"
        except:
            return locale.getdefaultlocale()[1]